# Librerias y constantes

In [1]:
#Librerias generales
import math, re
from decimal import Decimal, ROUND_HALF_UP
from datetime import datetime
import pytz                               # pip install pytz
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors          # pip install reportlab
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Table, TableStyle, Spacer
from reportlab.lib.units import inch

print("⚠️  Unidades obligatorias:")
print("    • Temperatura en °C")
print("    • Presión en hPa (el programa la convierte a mm Hg)")
print("    • Todas las lecturas de carga en nC\n")

# — Constantes de la cámara FC65-G —
N_Dw_Q0_cGy_per_nC = 4.819
k_Q_Q0             = 0.991024277
k_elec             = 1.000
T0_C, P0_mmHg      = 20.0, 760.0
PDD_default        = 67.33

⚠️  Unidades obligatorias:
    • Temperatura en °C
    • Presión en hPa (el programa la convierte a mm Hg)
    • Todas las lecturas de carga en nC



# Funciones

In [2]:
def input_float(prompt):
    while True:
        raw = input(prompt)
        try:
            if raw.strip() == "":
                raise ValueError
            return float(raw)
        except ValueError:
            print("❌ Introduce un número (entero o decimal).")

def input_yesno(prompt):
    while True:
        ans = input(prompt).strip().lower()
        if ans in ("si", "no"):
            return ans
        print("❌ Responde únicamente 'si' o 'no'.")

def input_letters(prompt):
    patron = r"[A-Za-zÁÉÍÓÚáéíóúÑñ ]+"
    while True:
        ans = input(prompt).strip()
        if re.fullmatch(patron, ans):
            return ans
        print("❌ Solo letras y espacios.")

def round_one_decimal(x):
    return float(Decimal(str(x)).quantize(Decimal('0.1'), ROUND_HALF_UP))

def k_tp(T_C, P_mmHg):
    return (273.2 + T_C) / (273.2 + T0_C) * (P0_mmHg / P_mmHg)

def k_pol(M300, Mminus300):
    return (abs(M300) + abs(Mminus300)) / (2 * M300)

def k_s(M300, M150):
    r = M300 / M150
    return 2.337 - 3.636 * r + 2.299 * r**2

def collect_readings_list(label):
    vals = [input_float(f"{label} lectura {i+1} (nC): ") for i in range(3)]
    while input_yesno(f"¿Quieres sustituir alguna {label}? (si/no): ") == "si":
        idx = int(input("   → ¿Cuál lectura (1-3)?: ")) - 1
        if idx not in (0, 1, 2):
            print("   Índice fuera de rango."); continue
        vals[idx] = input_float(f"   Nueva {label} lectura {idx+1} (nC): ")
    return vals

def make_centered_table_left(data):
    tbl = Table(data, hAlign="LEFT")
    tbl.setStyle(TableStyle([
        ('ALIGN',   (0,0), (-1,-1), 'CENTER'),
        ('VALIGN',  (0,0), (-1,-1), 'MIDDLE'),
        ('GRID',    (0,0), (-1,-1), 0.5, colors.black),
        ('BACKGROUND', (0,0), (-1,0), colors.lightgrey)
    ]))
    return tbl


# Datos de entrada

In [3]:
# 1. Datos generales
operator_name = input_letters("Nombre del usuario que realiza el cálculo: ")

# ── Captura y confirmación de temperatura y presión ──────────
T_C   = input_float("Temperatura (°C): ")
P_hPa = input_float("Presión (hPa): ")

while True:
    P_mmHg = P_hPa * 0.75006
    print(f"\nT = {T_C:.1f} °C | P = {P_hPa:.1f} hPa  ({P_mmHg:.1f} mm Hg)")
    if input_yesno("¿Valores correctos? (si/no): ") == "si":
        break                         # ► Confirmados, se continúa
    # — Corrección interactiva —
    while True:
        cual = input("   ¿Qué valor deseas cambiar? (T/P, 'salir' para volver): ").strip().lower()
        if cual == "t":
            T_C = input_float("   Nueva temperatura (°C): ")
        elif cual == "p":
            P_hPa = input_float("   Nueva presión (hPa): ")
        elif cual == "salir":
            break                    # sale al encabezado para mostrar de nuevo
        else:
            print("   Entrada no válida. Escribe 'T', 'P' o 'salir'.")
            continue


# 2. Lecturas iniciales
def collect_readings_list(label):
    valores = [input_float(f"{label} lectura {i+1} (nC): ") for i in range(3)]
    while input_yesno(f"¿Quieres sustituir alguna {label}? (si/no): ") == "si":
        idx = int(input("   → ¿Cuál lectura (1-3)?: ")) - 1
        if idx not in (0, 1, 2):
            print("   Índice fuera de rango."); continue
        valores[idx] = input_float(f"   Nueva {label} lectura {idx+1} (nC): ")
    return valores

readings_300  = collect_readings_list("+300 V")
readings_150  = collect_readings_list("+150 V")
readings_m300 = collect_readings_list("−300 V")

# 3. Función de resumen de lecturas
def show_summary():
    print("\n📊 Resumen de lecturas (nC):")
    print("{:<10} {:>10} {:>10} {:>10}".format("Voltaje","Lect 1","Lect 2","Lect 3"))
    print("{:<10} {:>10} {:>10} {:>10}".format(
          "+300 V", f"{readings_300[0]:.2f}", f"{readings_300[1]:.2f}", f"{readings_300[2]:.2f}"))
    print("{:<10} {:>10} {:>10} {:>10}".format(
          "+150 V", f"{readings_150[0]:.2f}", f"{readings_150[1]:.2f}", f"{readings_150[2]:.2f}"))
    print("{:<10} {:>10} {:>10} {:>10}".format(
          "−300 V", f"{readings_m300[0]:.2f}", f"{readings_m300[1]:.2f}", f"{readings_m300[2]:.2f}"))

# 4. Bucle de corrección de lecturas
while True:
    show_summary()
    if input_yesno("¿Deseas hacer otro cambio en las lecturas? (si/no): ") == "no":
        break
    volt = input("Voltaje a corregir (+300, +150, -300): ").strip()
    if volt not in ("+300", "+150", "-300"):
        print("Voltaje no válido. Usa +300, +150 o -300."); continue
    idx = int(input("Número de lectura a corregir (1-3): ")) - 1
    if idx not in (0, 1, 2):
        print("Índice fuera de rango."); continue
    nuevo = input_float("Nuevo valor (nC): ")

    if volt == "+300":
        readings_300[idx]  = nuevo
    elif volt == "+150":
        readings_150[idx]  = nuevo
    else:
        readings_m300[idx] = nuevo

# 5. Promedios definitivos
M300       = sum(readings_300)  / 3
M150       = sum(readings_150)  / 3
Mminus300  = sum(readings_m300) / 3
M300_initial = M300    # para reportes y PDF posteriores


Nombre del usuario que realiza el cálculo:  Luis Gutiérrez Melgarejo
Temperatura (°C):  22.1
Presión (hPa):  776.2



T = 22.1 °C | P = 776.2 hPa  (582.2 mm Hg)


¿Valores correctos? (si/no):  si
+300 V lectura 1 (nC):  10.77
+300 V lectura 2 (nC):  10.77
+300 V lectura 3 (nC):  10.77
¿Quieres sustituir alguna +300 V? (si/no):  no
+150 V lectura 1 (nC):  10.73
+150 V lectura 2 (nC):  10.73
+150 V lectura 3 (nC):  10.72
¿Quieres sustituir alguna +150 V? (si/no):  no
−300 V lectura 1 (nC):  10.79
−300 V lectura 2 (nC):  10.79
−300 V lectura 3 (nC):  10.80
¿Quieres sustituir alguna −300 V? (si/no):  no



📊 Resumen de lecturas (nC):
Voltaje        Lect 1     Lect 2     Lect 3
+300 V          10.77      10.77      10.77
+150 V          10.73      10.73      10.72
−300 V          10.79      10.79      10.80


¿Deseas hacer otro cambio en las lecturas? (si/no):  no


# Calculos y bucle de correcciónes

In [4]:
# Factores de corrección
ktp_val  = k_tp(T_C, P_mmHg)
kpol_val = k_pol(M300, Mminus300)
ks_val   = k_s(M300, M150)
corrections_history = []

print("\n📌 Factores de corrección y resultados iniciales:")
print(f"  k_tp     = {ktp_val:.4f}")
print(f"  k_pol    = {kpol_val:.4f}")
print(f"  k_s      = {ks_val:.4f}")
print(f"  k_elec   = {k_elec:.4f}")
print(f"  k_Q,Q0   = {k_Q_Q0:.6f}")

iteracion = 0
while True:
    # — Cálculos —
    M_q   = M300 * ktp_val * kpol_val * ks_val * k_Q_Q0 * k_elec
    D_ref = M_q * N_Dw_Q0_cGy_per_nC
    Dw    = D_ref * 100 / PDD_default
    err_rel = (Dw - 100)     # error relativo en %

    # — Mostrar resultados —
    print(f"  M_q      = {M_q:.3f} nC")
    print(f"  D_ref    = {D_ref:.3f} cGy")
    print(f"  D_w,Q    = {Dw:.2f} cGy/UM")
    print(f"  Error rel (%) = {err_rel:+.2f}")

    # — Guardar en historial —
    corrections_history.append({
        "iteration": iteracion,
        "M300":   M300,
        "M_q":    M_q,
        "Dref":   D_ref,
        "Dw":     Dw,
        "error":  err_rel
    })

    # — Preguntar si habrá corrección —
    if input_yesno("¿Se van a hacer ajustes? (si/no): ") == "no":
        if iteracion == 0:
            print("\n✅ Debido a que no hay correcciones, el proceso ha finalizado.")
        break

    print(f"\n➡️ Indica al ingeniero introducir Dw,Q = {round_one_decimal(Dw):.1f} cGy/MU\n")
    M300 = sum(collect_readings_list(f"CORRECCIÓN #{iteracion+1} +300 V")) / 3
    iteracion += 1



📌 Factores de corrección y resultados iniciales:
  k_tp     = 1.3148
  k_pol    = 1.0011
  k_s      = 1.0039
  k_elec   = 1.0000
  k_Q,Q0   = 0.991024
  M_q      = 14.103 nC
  D_ref    = 67.963 cGy
  D_w,Q    = 100.94 cGy/UM
  Error rel (%) = +0.94


¿Se van a hacer ajustes? (si/no):  no



✅ Debido a que no hay correcciones, el proceso ha finalizado.


# Generación de PDF

In [6]:
# 🟢 Celda 5 – Reporte TRS-398 con membrete + subíndices en tabla
# Requiere:  pip install reportlab

from reportlab.platypus import (
    BaseDocTemplate, PageTemplate, Frame, Paragraph, Spacer,
    Table, TableStyle, PageBreak
)
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import cm
from reportlab.lib import colors
from datetime import datetime
import pytz, re, os
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.platypus import (
    BaseDocTemplate, PageTemplate, Frame, Paragraph, Spacer,
    Table, TableStyle, PageBreak,
    FrameBreak,            # ← nuevo
    KeepTogether           # ← nuevo (útil si luego quieres proteger tablas grandes)
)
pdfmetrics.registerFont(TTFont('Arial', r'C:\Windows\Fonts\arial.ttf'))
path_regular = r"C:\Windows\Fonts\arial.ttf"      # Arial Regular
path_bold    = r"C:\Windows\Fonts\ARLRDBD.ttf"    # Arial Bold (nombre real del archivo)
pdfmetrics.registerFont(TTFont('Arial', r'C:\Windows\Fonts\arial.ttf'))
pdfmetrics.registerFont(TTFont('ArialBold',  path_bold))
pdfmetrics.registerFontFamily(
    'Arial',
    normal='Arial',
    bold='ArialBold',
)
# --------------------------------------------------------------------------
# RUTA DEL MEMBRETE
BACKGROUND_IMG = r"C:\JN\Fondo.jpg"   # ← ajusta si cambia la ruta
# --------------------------------------------------------------------------

def add_background(c, doc):
    c.drawImage(BACKGROUND_IMG, 0, 0, width=letter[0], height=letter[1])

# ---------- Nombre del PDF ----------
def ask_name(prompt="Nombre del PDF (sin extensión): "):
    patron = r"[A-Za-zÁÉÍÓÚáéíóúÑñ0-9 _-]+"
    while True:
        n = input(prompt).strip()
        if re.fullmatch(patron, n):
            return n.replace(" ", "_")

pdf_path = f"{ask_name()}.pdf"

# ---------- Márgenes ----------
LM = RM = 3   * cm
TM = BM = 2.5 * cm
usable_height = letter[1] - TM - BM   # alto del área útil en la página


# ---------- Estilos ----------
styles = getSampleStyleSheet()

base_kwargs = dict(fontName='Arial', fontSize=12, leading=16)

#  cuerpo normal
styles.add(ParagraphStyle('Body', fontName='Arial', fontSize=12, leading=16))

#  cuerpo justificado
styles.add(ParagraphStyle('BodyJ', parent=styles['Body'],
                          alignment=4))

#  cuerpo centrado
styles.add(ParagraphStyle('BodyCenter', parent=styles['Body'],
                          alignment=1))

#  título alineado a la izquierda (tamaño también 12 pt)
styles.add(ParagraphStyle('TitleL',
                          parent=styles['Body'],
                          fontName='ArialBold',  # usa la fuente bold registrada
                          alignment=0))

#  subtítulo (equivalente a H2Left) también 12 pt

styles.add(ParagraphStyle('H2Left',
                          parent=styles['Body'],
                          fontName='ArialBold',
                          alignment=0))
# ---------- Plantilla con fondo ----------
# ---------- Plantilla con fondo + 2 frames ----------
# 1) crea el documento
doc = BaseDocTemplate(pdf_path, pagesize=letter,
                      leftMargin=LM, rightMargin=RM,
                      topMargin=TM, bottomMargin=BM)

# 2) ahora que doc existe, usa sus dimensiones
BOTTOM_BAND = 3 * cm

MAIN_FRAME = Frame(
    LM, BM + BOTTOM_BAND,
    doc.width,
    doc.height - BOTTOM_BAND,
    id="main_frame"
)

SIGN_FRAME = Frame(
    LM, BM,
    doc.width,
    BOTTOM_BAND,
    id="sign_frame"
)

# 3) plantilla de página
doc.addPageTemplates([
    PageTemplate(id="BG",
                 frames=[MAIN_FRAME, SIGN_FRAME],
                 onPage=add_background)
])
story = []

# ------------------ ENCABEZADO ------------------
story.append(Spacer(1, 1.2 * cm))  # despeja logos
now = datetime.now(pytz.timezone("America/Mexico_City")).strftime("%d/%m/%Y %H:%M")

story += [
    Paragraph(f"Realizado por: <b>{operator_name}</b>", styles["BodyJ"]),
    Paragraph(f"Fecha: <b>{now}</b>", styles["BodyJ"]),
    Spacer(1, 0.3 * cm),
    Paragraph(
        ("El proceso de dosimetría se realizó siguiendo las recomendaciones "
         "del protocolo <b>TRS-398 (Rev 1) de 2024</b>. Para el acelerador "
         "<b>Elekta Infinity</b> con N.º de serie 156921, se consideró un valor de"
         "TPR<sub>20,10</sub> = <b>0.681</b> y PDD<sub>zmax</sub> = <b>67.33 %</b>."),
        styles["BodyJ"]),
    Paragraph (
        ("El equipo de dosimetría utilizado para esta calibración se lista acontinuación"),
        styles["BodyJ"]),
    Spacer(1, 0.5 * cm)
]

# ------------------ EQUIPO DOSIMÉTRICO ------------------
equipos = [
    "Tanque de dosimetría, SmartScan de la marca IBA con N.º de serie 24569)",
    "Electrómetro, modelo Dose 1 de la marca IBA con N.º de serie 14866",
    "Barómetro de la marca Testo 511",
    "Termómetro modelo P700 de la marca Dostmann",
    ("Cámara de ionización tipo cilíndrica modelo FC65-G de la marca IBA con N.º de serie 5574 y factor de"
     "calibración N<sub>D,w,Q0</sub> = <b>4.819 cGy/nC</b>")
]
for item in equipos:
    story.append(Paragraph("• " + item, styles["BodyJ"]))
story.append(Spacer(1, 0.3 * cm))

# ------------------ FACTORES DE CORRECCIÓN ------------------
story.append(Paragraph("Factores de corrección por cantidades de influencia",
                       styles["TitleL"]))
story.append(Spacer(1, 0.3 * cm))
tbl_k_data = [
    [Paragraph("<b>Factor</b>", styles["BodyCenter"]),
     Paragraph("<b>Valor</b>",  styles["BodyCenter"])],
    [Paragraph("k<sub>tp</sub>",   styles["BodyCenter"]), f"{ktp_val:.4f}"],
    [Paragraph("k<sub>pol</sub>",  styles["BodyCenter"]), f"{kpol_val:.4f}"],
    [Paragraph("k<sub>s</sub>",    styles["BodyCenter"]), f"{ks_val:.4f}"],
    [Paragraph("k<sub>Q,Q0</sub>", styles["BodyCenter"]), f"{k_Q_Q0:.6f}"],
    [Paragraph("k<sub>elec</sub>", styles["BodyCenter"]), f"{k_elec:.4f}"]
]

tbl_k = Table(tbl_k_data, hAlign="CENTER", colWidths=[6*cm, 4*cm])  # ← centrado aquí
tbl_k.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
    ('ALIGN',      (0, 0), (-1, -1), 'CENTER'),
    ('VALIGN',     (0, 0), (-1, -1), 'MIDDLE'),
    ('GRID',       (0, 0), (-1, -1), 0.5, colors.black)
]))
story += [tbl_k, Spacer(1, 0.5 * cm)]

# ------------------  LECTURAS INICIALES ------------------
story.append(Paragraph("Primera verificación de la dosis", styles["TitleL"]))

# Datos de lectura
story.append(Spacer(1, 0.3 * cm))
lect_tbl_data = [
    # Fila 0 – encabezados principales (celdas 1 y 2 unidas)
    [Paragraph("<b>Lecturas</b>", styles["BodyCenter"]),
     Paragraph("<b>Voltaje utilizado</b>", styles["BodyCenter"]),
     "", "", ""],          # ← columnas “vacías” para completar el span
    # Fila 1 – sub-encabezados de voltaje
    ["", "+300 V", "+150 V", "−300 V", ""],  # la 1.ª celda vacía debajo de “Lecturas”
    # Filas 2-4 – Q1…Q3
    [Paragraph("Q<sub>1</sub> [nC]", styles["BodyCenter"]),
     f"{readings_300[0]:.2f}", f"{readings_150[0]:.2f}", f"{readings_m300[0]:.2f}", ""],
    [Paragraph("Q<sub>2</sub> [nC]", styles["BodyCenter"]),
     f"{readings_300[1]:.2f}", f"{readings_150[1]:.2f}", f"{readings_m300[1]:.2f}", ""],
    [Paragraph("Q<sub>3</sub> [nC]", styles["BodyCenter"]),
     f"{readings_300[2]:.2f}", f"{readings_150[2]:.2f}", f"{readings_m300[2]:.2f}", ""],
    # Fila 5 – promedio
    [Paragraph("Q<sub>promedio</sub> [nC]", styles["BodyCenter"]),
     f"{sum(readings_300)/3:.2f}",
     f"{sum(readings_150)/3:.2f}",
     f"{sum(readings_m300)/3:.2f}",
     ""]
]

lect_tbl = Table(lect_tbl_data, hAlign="CENTER", colWidths=[6*cm, 4*cm])  # ← centrado aquí

# Anchuras de columnas (ajústalas si quieres un ancho total distinto)
col_w = [5*cm, 3*cm, 3*cm, 3*cm, 0.1*cm]  # la última (0.1 cm) es un “dummy” casi invisible

lect_tbl = Table(lect_tbl_data, hAlign="LEFT", colWidths=col_w)

# Estilos y SPANs
lect_tbl.setStyle(TableStyle([
    # Fondo gris para la fila 0 y fila 1
    ('BACKGROUND', (0,0), (-1,1), colors.lightgrey),

    # ── SPANS ───────────────────────────────
    ('SPAN', (0,0), (0,1)),      # ‘Lecturas’ ocupa 2 filas
    ('SPAN', (1,0), (4,0)),      # ‘Voltaje utilizado’ ocupa 4 columnas

    # Opcional: ocultar la 5.ª columna “dummy”
    ('SPAN', (4,1), (4,5)),      # la fusionamos para que no parta filas

    # Centrados y bordes
    ('ALIGN',  (0,0), (-1,-1), 'CENTER'),
    ('VALIGN', (0,0), (-1,-1), 'MIDDLE'),
    ('GRID',   (0,0), (-1,-1), 0.5, colors.black)
]))

story += [lect_tbl, Spacer(1, 0.5*cm), PageBreak()]

# ------------------  RESULTADOS INICIALES ------------------
story.append(Spacer(1, 1.2 * cm))  # despeja logos / membrete
story.append(Paragraph("<b>Resultados para la primera verificación</b>",
                       styles["TitleL"]))
story.append(Spacer(1, 0.3 * cm))
init_data = [
    # etiqueta            │ valor
    [Paragraph("M<sub>q</sub> [nC]", styles["BodyCenter"]),
     Paragraph(f"<b>{corrections_history[0]['M_q']:.3f}</b>", styles["BodyCenter"])],

    [Paragraph("D<sub>ref</sub> [cGy]", styles["BodyCenter"]),
     Paragraph(f"<b>{corrections_history[0]['Dref']:.3f}</b>", styles["BodyCenter"])],

    [Paragraph("<b>D<sub>w,Q</sub> [cGy/MU]</b>", styles["BodyCenter"]),
     Paragraph(f"<b>{corrections_history[0]['Dw']:.2f}</b>", styles["BodyCenter"])],

    [Paragraph("<b>Error relativo [%]</b>", styles["BodyCenter"]),
     Paragraph(f"<b>{((corrections_history[0]['Dw'] - 100) / 100)*100:.2f} %</b>",
               styles["BodyCenter"])]
]

#  colWidths = [ancho etiqueta, ancho valor]
table2 = Table(init_data,
               colWidths=[5 * cm, 2.8 * cm],   # ← hazlos más pequeños si quieres
               hAlign="CENTER")                # o "LEFT" / "RIGHT"

table2.setStyle(TableStyle([
    #('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),  # cabecera gris
    ('ALIGN',      (0, 0), (-1, -1), 'CENTER'),         # texto centrado
    ('VALIGN',     (0, 0), (-1, -1), 'MIDDLE'),
    ('GRID',       (0, 0), (-1, -1), 0.5, colors.black),
        # Sombreado amarillo en las filas de Dw y error
    ('BACKGROUND', (0, 2), (-1, 2), colors.yellow),
    ('BACKGROUND', (0, 3), (-1, 3), colors.yellow)
]))

story.append(table2)
story.append(Spacer(1, 0.5 * cm))  # deja 1 cm de margen al bloque siguiente


# ------------------  CORRECCIONES EXTRA ------------------
if len(corrections_history) > 1:
    story.append(PageBreak())
    story.append(Spacer(1, 4*cm))
    story.append(Paragraph(
        ("Debido a que la dosis necesitó correcciones, se muestran las "
         "iteraciones necesarias."), styles["Body"]))
    for corr in corrections_history[1:]:
        story.append(Spacer(1, 0.3*cm))
        story.append(Paragraph(
            f"Valores de carga para la iteración #{corr['iteration']}",
            styles["TitleL"]))
        carga_tbl = Table([
            ["Q1 [nC]", "Q2 [nC]", "Q3 [nC]", "Q̅ [nC]"],
            [f"{corr['M300']:.2f}", "", "", f"{corr['M300']:.2f}"]
        ], hAlign="LEFT")
        carga_tbl.setStyle(TableStyle([
            ('BACKGROUND',(0,0),(-1,0),colors.lightgrey),
            ('ALIGN',(0,0),(-1,-1),'CENTER'),
            ('GRID',(0,0),(-1,-1),0.5,colors.black)
        ]))
        story.append(carga_tbl)
        res_tbl = Table([
            ["Lectura corregida [nC]",    f"{corr['M_q']:.3f}"],
            ["Dosis en z<sub>ref</sub> [cGy]", f"{corr['Dref']:.3f}"],
            ["<b>Dosis en z<sub>max</sub></b> [cGy]", f"{corr['Dw']:.2f}"],
            ["Error relativo",        f"{corr['error']:+.2f}"]
        ], hAlign="LEFT", colWidths=[9*cm, 4*cm])
        res_tbl.setStyle(TableStyle([
            ('ALIGN',(0,0),(-1,-1),'CENTER'),
            ('GRID',(0,0),(-1,-1),0.5,colors.black)
        ]))
        story.append(Spacer(1, 0.2*cm))
        story.append(res_tbl)

# ------------------  CONCLUSIÓN Y FIRMA ------------------

final = corrections_history[-1]
#story.append(Spacer(1, 1*cm))
story.append(Paragraph(
    (f"Derivado de la calibración se determinó que la dosis absorbida en "
     f"z<sub>max</sub> es de <b>{final['Dw']:.2f} cGy/UM</b> con error "
     f"relativo de <b>{final['error']:+.2f}%</b> cuyo valor esta dentro de la tolerancia del ±3 % que se "
     f"recomienda de acuerdo con normas internacionales."),
    styles["BodyJ"]))

story.append(FrameBreak())
story.append(Paragraph(
    "Verificado por: _______________________________",
    styles["BodyCenter"]))      # o styles["Body"]

# ------------------  GENERAR PDF ------------------
doc.build(story)
print(f"✅ PDF generado correctamente en: {pdf_path}")


Nombre del PDF (sin extensión):  1


LayoutError: Flowable <Table@0x2631CA887A0 6 rows x 5 cols(tallest row 22)> with cell(0,0) containing
'<Paragraph at 0x2631ca8bdd0>Lecturas'(399.6850393700787 x 128), tallest cell 22.0 points,  too large on page 1 in frame 'sign_frame'(429.9212598425197 x 73.03937007874015*) of template 'BG'